# Вычислительная семантика и классификация текстов

1) Семантическая близость. Модели CBOW и Skip-Gram.  
2) Ембеддинги Word2Vec и Glove.  
3) Чат бот c word2vec.

### 1) Семантическая близость. Модели CBOW и Skip-Gram.

В английском языке насчитывается около 13 миллионов уникальных токенов. Но насколько уникальны смыслы этих слов? Насколько близки по смыслу слова motel и hotel? 
Мы уже рассмотрели, как можно представить каждое слово в виде вектора в пространстве большой размерности. Однако на самом деле для эффективного кодирования достаточно использовать пространство смыслов, которое имеет намного меньшую размерность.  
Семантика слова определяет его пренадлежность к определенной смысловой категории: время, количество, род, назначение и т.д.

<img src='imgs/semantics.jpg' width=400>

Определить вектор смыслов для 13 миллионов токенов задача довольно сложная. Вручную это сделать практически невозможно, поэтому лучше поручить это компьютеру. Предположим, что близкие по смыслу слова часто встречаются в одном предложении (документе). Например, слова "банки", "облигации", "акции", "деньги" вероятно будут появляться вместе. Но "банки", "осьминог", "банан" и "хоккей", вероятно, не будут последовательно появляться вместе в одном документе. 

<img src="imgs/cooccurence.jpg" width=640>

Используем этот факт для построения матрицы совпадений (co-occurence). Перебирая миллиарды документов, каждый раз, когда слово i появляется в документе j, мы добавляем единицу к записи $X_{i, j}$. В результате получается очень большая матрица  размером $R^{V × V}$  

<img src="imgs/comatrix.jpg" width=400>


Проблема заключается в том, что матрица размером $10^7 \cdot 10^7$ просто не поместится в памяти компьютера.  
Используя алгоритм сингулярного разложения, можно привести матрицу совпадений к виду 
$$M=U\Sigma V^{*}$$ 
где $\Sigma$  — матрица размера $m\times n$ с неотрицательными элементами, у которой элементы, лежащие на главной диагонали — это сингулярные числа (а все элементы, не лежащие на главной диагонали, являются нулевыми),  
матрицы $U$ (порядка m) и $V$ (порядка n) — это две унитарные матрицы, состоящие из левых и правых сингулярных векторов соответственно  
$V^*$ — это сопряжённо-транспонированная матрица к $V$.  

Такое преобразование позволяет уменьшить размерность матрицы за счет небольшой потери информации о взаимной частоте появления слов. Из-за сложностей с вычислением сингулярного разложения, на практике этот метод имеет ряд ограничений. В 2013 году Т.Миколов предложил алгоритм Word2Vec для вычисления ембеддингов (векторов меньшей размерности, смысловых векторов), который включает в себя две модели: CBOW и Skip-Gramm. 

### CBOW (Continues Bag Of Words)

Позволяет предсказывать центральное слово по его контексту. Например, для контекста ("банк", "облигации") самым вероятным словом будет "деньги". Контекст кодируется как OneHot вектор $x$ и является входом для модели. Выходом является  OneHot вектор  $y^V$, в котором закодировано центральное слово. Обучается модель на множестве пар векторов $(x, y)$, которые берутся при переборе большого количества документов. На каждом шаге вычисляется CrossEntropy loss и параметры модели обновляются, пока не достигнут оптимальных значений.    

<img src='imgs/CBOW.jpg'>

Вместо одной большой матрицы $W$, в модели используются две матрицы $U$ и $V$. Как правило, размер вектора эмбеддингов на два порядка меньше, чем OneHot: $10^3$ вместо $10^5$. Во-первых, это позволяет сэкономить память: $2 \cdot 10^8$ вместо $10^{10}$. Во-вторых, обучив модель на большом корпусе текста, мы можем сложить матрицы $U + V.T$ и использовать результат непосредственно для получения вектора эмбеддингов из OneHot вектора для любого контекста. 

### Skip Gram 

Позволяет предсказывать распределение вероятностей контекста по заданному слову. Например, для слова "деньги" самым вероятным контекстом будет ("банк", "облигации"). Слово кодируется как OneHot вектор $x$ и является входом для модели. Выходом является  OneHot вектор  $y$, в котором закодированы слова контекста. Обучается модель на множестве пар векторов $(x, y)$, которые берутся при переборе большого количества документов. На каждом шаге вычисляется CrossEntropy loss и параметры модели обновляются, пока не достигнут оптимальных значений.    

<img src='imgs/skipgrams.jpg'>

Так же как и для модели CBOW, обучив модель на большом корпусе текста, мы можем сложить матрицы $U + V.T$ и использовать результат непосредственно для получения вектора эмбеддингов из OneHot вектора для любого слова. 

### 2) Ембеддинги Word2Vec и Glove.

Gensim (Generate Similar) - это бесплатная библиотека Python с открытым исходным кодом для представления документов в виде семантических векторов, максимально эффективно (с точки зрения компьютера) и безболезненно (с точки зрения человека).  
Gensim предназначен для обработки неструктурированных текстовых файлов с использованием  алгоритмов машинного обучения без учителя.

In [1]:
import gensim.downloader
word2vec_ru = gensim.downloader.load('word2vec-ruscorpora-300')

Словарь предобученной модели содержит слова и аннотации части речи к ним (POS, part of speech tagging) 

In [2]:
word2vec_ru.key_to_index

{'весь_DET': 0,
 'человек_NOUN': 1,
 'мочь_VERB': 2,
 'год_NOUN': 3,
 'сказать_VERB': 4,
 'время_NOUN': 5,
 'говорить_VERB': 6,
 'становиться_VERB': 7,
 'знать_VERB': 8,
 'самый_DET': 9,
 'дело_NOUN': 10,
 'день_NOUN': 11,
 'жизнь_NOUN': 12,
 'рука_NOUN': 13,
 'очень_ADV': 14,
 'первый_ADJ': 15,
 'давать_VERB': 16,
 'новый_ADJ': 17,
 'слово_NOUN': 18,
 'иметь_VERB': 19,
 'большой_ADJ': 20,
 'идти_VERB': 21,
 'глаз_NOUN': 22,
 'место_NOUN': 23,
 'лицо_NOUN': 24,
 'видеть_VERB': 25,
 'хотеть_VERB': 26,
 'понимать_VERB': 27,
 'должный_ADJ': 28,
 'работа_NOUN': 29,
 'каждый_DET': 30,
 'друг_NOUN': 31,
 'голова_NOUN': 32,
 'дом_NOUN': 33,
 'оставаться_VERB': 34,
 'сторона_NOUN': 35,
 'начинать_VERB': 36,
 'думать_VERB': 37,
 'хорошо_ADV': 38,
 'жить_VERB': 39,
 'стоять_VERB': 40,
 'спрашивать_VERB': 41,
 'сделать_VERB': 42,
 'выходить_VERB': 43,
 'последний_ADJ': 44,
 'русский_ADJ': 45,
 'сила_NOUN': 46,
 'получать_VERB': 47,
 'какой-то_DET': 48,
 'хороший_ADJ': 49,
 'случай_NOUN': 50,
 'во

In [3]:
word2vec_ru.most_similar('дом_NOUN')

[('дома_ADV', 0.6836265921592712),
 ('домик_NOUN', 0.6716411113739014),
 ('особняк_NOUN', 0.6681575775146484),
 ('квартира_NOUN', 0.6364003419876099),
 ('флигель_NOUN', 0.6331559419631958),
 ('двухэтажный_ADJ', 0.5991461873054504),
 ('полукаменный_ADJ', 0.5857471823692322),
 ('мезонин_NOUN', 0.5743653774261475),
 ('усадьба_NOUN', 0.5727247595787048),
 ('изба_NOUN', 0.5710321664810181)]

In [4]:
word2vec_ru.most_similar(positive=['банк_NOUN', 'деньги_NOUN'], topn=1)

[('кредит_NOUN', 0.6994256377220154)]

Для получения аннотаций используется библиотека spacy

In [5]:
!python -m spacy download ru_core_news_md

     |████████████████████████████████| 41.9 MB 3.1 MB/s eta 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_md')


In [6]:
import spacy
nlp = spacy.load('ru_core_news_md')

In [7]:
text = 'мой дядя самых честных правил'
document = nlp(text)
for token in document:
    try:
        similars = word2vec_ru.most_similar(f'{token.lemma_}_{token.pos_}', topn=3)
        print(token.lemma_)
        print(similars)
    except KeyError:
        pass

дядя
[('племянник_NOUN', 0.7086743116378784), ('дедушка_NOUN', 0.638724684715271), ('тетка_NOUN', 0.629277229309082)]
честной
[('честный_ADJ', 0.6214777231216431), ('подлячка_NOUN', 0.45800015330314636), ('добросердый_ADJ', 0.45189282298088074)]
правило
[('соблюдение_NOUN', 0.5640134811401367), ('закон_NOUN', 0.5540075302124023), ('устав_NOUN', 0.5533488392829895)]


### 3)  Чат бот c word2vec.

In [8]:
def get_answer(text):
    document = nlp(text)
    doc_tok = []

    for token in document:
        t = f'{token.lemma_}_{token.pos_}'
        if t in word2vec_ru.key_to_index.keys():
            doc_tok.append(t)

    if len(doc_tok) > 0:
        sims = word2vec_ru.most_similar(doc_tok, topn=3)
        sims_words = [w[0].split('_')[0] for w in sims]
        return 'Я знаю близкие по смыслу слова: ' + ' '.join(sims_words)
    else:
        return 'Я не знаю таких слов.'

In [9]:
import telebot

with open('texts/tlg_token.txt') as f:
    TOKEN = f.read()

bot = telebot.TeleBot(TOKEN) 

@bot.message_handler(content_types=['text'])
def send_echo(message):
    answer = get_answer(message.text)
    
    bot.send_message(message.chat.id, answer)

bot.polling(none_stop=True)

Литература:  
- Хобсон NLP in actioin, Глава 6  
- https://radimrehurek.com/gensim/auto_examples/core/run_core_concepts.html